In [ ]:
# -- configuring artifact registry
# -- building a custom image with bigquery and pushing
# -- adding worker pool
# -- optional: deploying an image which is pre-built
# -- optional: adding service account via labels
# -- setting default worker pool to custom pool
# -- scaling down `default-worker-pool`